# Imports

In [1]:
!pip install ta==0.3.8

In [ ]:
import pandas
import numpy
import datetime
from datetime import timedelta
from enum import Enum
import argparse
from ta import *

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import random
import copy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal, Categorical



# Classes

In [0]:
class Singleton(object):
    def __new__(cls, *args, **kw):
        if not hasattr(cls, '_instance'):
            orig = super(Singleton, cls)
            cls._instance = orig.__new__(cls, *args, **kw)
        return cls._instance


      
      
#Config enviromnet elements       
class EnvironmentConfiguration():
    Symbol = ""
    TrainStartDate = datetime.datetime.now()
    TrainEndDate = datetime.datetime.now()
      
#Data Of every Frame  
class StockData:
    Symbol = ""
    IndexDate = ""
    Close = 0
    Low = 0
    High = 0
    Volume = 0
    volatility_atr=0
    trend_macd=0
    momentum_rsi=0


class StockDataList(Singleton):
    df = pandas.DataFrame()

    
# fill the data    
class State():
    HoldStock = False
    StockDataFrame = pandas.DataFrame()
    IndexDate = datetime.datetime.now()
    stockData = StockData()
    lastBuyStockData = StockData()
    pastStockData = StockData()

    def setDataFrame(self, indexDate):
        self.IndexDate = indexDate
        StockDatalist = StockDataList()
        StartDate = self.IndexDate 
        
        df = StockDatalist.df
        df = df[df['Date'] <= self.IndexDate]
        df = df[df['Date'] >= StartDate]
        self.StockDataFrame = df

        df2 = df[df['Date'] == self.IndexDate]
        if len(df) > 0:
            nstockdata = StockData()
            nstockdata.Close = df2['Close'].values[0]
            nstockdata.High = df2['High'].values[0]
            nstockdata.Volume = df2['V'].values[0]
            nstockdata.IndexDate = df2['Date'].values[0]
            nstockdata.Low = df2['Low'].values[0]
            nstockdata.Open = df2['Open'].values[0]
            nstockdata.volatility_atr7  = df2['ATR7'].values[0]
            nstockdata.volatility_atr14 = df2['ATR14'].values[0]
            nstockdata.volatility_atr21 = df2['ATR21'].values[0]
            nstockdata.volatility_atr32 = df2['ATR32'].values[0]
            
            
            nstockdata.trend_macd = df2['MACD'].values[0]
            nstockdata.momentum_rsi = df2['RSI'].values[0]
            
            
            self.stockData = nstockdata




#  Action type
class ActionType(Enum):
    Buy = -1
    Stay = 0
    Sell = 1




class Reward:
    TotalPoint = 0
    LastState = State()
    NextAction = ActionType



# frame for rewards
class PreviousRewards():
    name = "Rewards"
    dty = {
        "TotalPoint": "float64",
        "LastBuyDate": "datetime64",
        "Profits": "float64"
    }
    Rewards = pandas.DataFrame(columns=dty.keys())
    profit=0

 
    def __init__(self):
        self.Rewards.index.names = ['index']


 

  

    def Append(self, pastState, current_state, actionType):
        printText = ""
        reward = Reward()
        reward.LastState = copy.deepcopy(current_state)
        reward.NextAction = actionType
        reward.TotalPoint, ptext,profit = self.Calculate_reward(pastState, current_state, actionType)
      
        if current_state.lastBuyStockData is None:
            LastBuyDate = None
        else:
            LastBuyDate = current_state.lastBuyStockData.IndexDate

        row = [
               reward.TotalPoint,
               LastBuyDate,
               profit
               ]
        self.Rewards.loc[len(self.Rewards)] = row
        return reward, printText


        


        
        
        
        
        
        
        
       
        
#  reward Part    
        
    # reward for after buy action     
    def calculate_stay_reward(self,holdStock,currentState,pastState):
       rewardPoint=0
       if holdStock:  # stok Varsa
          currentStockData = copy.deepcopy(currentState.stockData)        
          rewardPoint = (currentStockData.Close - pastState.stockData.Close)
          if rewardPoint > 0:
            rewardPoint=0.1
          else:
            rewardPoint=-0.1
              
       else:  # Stok yoksa
           rewardPoint = -0.00
           
       return rewardPoint  
     
      
      
        
    
    def Calculate_reward(self, pastState, currentState,actionType):
        rewardType=''
        printText = ""
        
        
        if (currentState.HoldStock == True and actionType == ActionType.Buy):
            rewardPoint=self.calculate_stay_reward(currentState.HoldStock,currentState,pastState)
            
        elif (currentState.HoldStock == False and actionType == ActionType.Sell):
            rewardPoint = -0.01 
            
            
        else:

            if actionType == ActionType.Sell:
                buyStockData = copy.deepcopy(currentState.lastBuyStockData)
                sellStockData = copy.deepcopy(currentState.stockData)

                self.profit=self.profit + (sellStockData.Close - buyStockData.Close)
              
              
                
                profit = (sellStockData.Close - buyStockData.Close)

                
                if self.holdSteps!=0:
                    rewardPoint = profit 
                else:
                    rewardPoint=0.0
                   
                print(
                      'dayprofit=',sellStockData.Close-buyStockData.Close,'self.profit=',self.profit,
                      'BuyDate=',str(buyStockData.IndexDate)[2:10],str(buyStockData.IndexDate)[11:16] ,'SellDate=',str(sellStockData.IndexDate)[2:10],str(sellStockData.IndexDate)[11:16],
                      'buyPrice=',buyStockData.Close,'sellPrice=',sellStockData.Close,
                      'reward=',rewardPoint
                      )    
                if rewardPoint>0:
                  rewardPoint=0.5
                else:
                  rewardPoint=-0.5
                
            elif actionType == ActionType.Stay:
             
              rewardPoint=self.calculate_stay_reward(currentState.HoldStock,currentState,pastState)

            else:  # Buy
                self.holdSteps=0
               
                rewardPoint = 0.00

        rewardscaled = rewardPoint * 250
#         print('rewardscaled',rewardscaled)
        return rewardscaled, printText,self.profit






# Enviroment

In [0]:

# enviroment class to handle data and return every step
class Environment:
    config = EnvironmentConfiguration()
    Data = StockDataList()  
    CurrentState = State()
    pastState= State()

    Info = ""
    IterationID = 0 
    Rewards = PreviousRewards()
    stepCounter = 0
    render_on = 0
    EnvironmentID = 0
   
#reset Enviroment to start From Start Date of data
    def reset(self, IterationNumber):
        mindate = self.Data.df['Date'].min()
        self.CurrentState = State()
        self.pastState = State()

        self.CurrentState.setDataFrame(mindate)
        self.Info = ""
        self.IterationID = IterationNumber
        self.Rewards = PreviousRewards()
        self.stepCounter = 0

#     intialize Enviroment First Step and config    
    def initializeEnv(self, config): 
        self.EnvironmentID = int(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
        self.config = config
        self.Data = self.getData(config.DataLocation, config.Symbol, config.TrainStartDate, config.TrainEndDate)
        mindate = self.Data.df['Date'].min()
        self.CurrentState.setDataFrame(mindate)
        
        
#   return next State,Reward,Done,and summary   
    def step(self , actionType):
        done = False
        reward, ptext = self.Rewards.Append(self.pastState,self.CurrentState, actionType)
        self.Info = "Next Step in State " + str(self.CurrentState) + " with Action " + str(actionType) + " and Gain Reward " + str(reward.TotalPoint)
        self.pastState=copy.deepcopy(self.CurrentState)
        nextState = State() 
        nextState.HoldStock = self.CurrentState.HoldStock
        nextState.lastBuyStockData = copy.deepcopy(self.CurrentState.lastBuyStockData) 

        if actionType == ActionType.Sell:
            if self.CurrentState.HoldStock==True:
                done=True
            nextState.HoldStock = False
            nextState.lastBuyStockData = StockData()
            nextState.Quantity = 0
        elif actionType == ActionType.Buy:
            if self.CurrentState.HoldStock==False:
                nextState.HoldStock = True
                nextState.lastBuyStockData = copy.deepcopy(self.CurrentState.stockData)
                nextState.Quantity = 1


        else: #Stay
            nextState.lastBuyStockData = copy.deepcopy(self.CurrentState.lastBuyStockData)
        tempDF= str(self.Data.df['Date'].values[0]) + ' ' + str(self.Data.df['Time'].values[0])
        DF= pandas.to_datetime(str(self.Data.df['Date'].values[0]) + ' ' + str(self.Data.df['Time'].values[0]))
      





        newIndexDate = (self.Data.df[self.Data.df['Date']  > self.CurrentState.IndexDate])['Date'].min()

     

        if  pandas.isnull(newIndexDate): 
            done = True
        nextState.IndexDate = newIndexDate
        nextState.setDataFrame(newIndexDate)
        
        self.CurrentState = copy.deepcopy(nextState)
        self.stepCounter = self.stepCounter  + 1
        return self.CurrentState, reward, done, self.Info
      
      
      
      
      
    def getDataPointCount(self):
        return len(self.Data.df)
      
      
      
      
      
    # import csv file
    def getData(self, datalocation, Symbol='', StartDate = datetime.datetime.strptime('2001/1/1 00:00', "%Y/%m/%d %H:%M"), EndDate=datetime.datetime.strptime('2101/1/1 00:00', "%Y/%m/%d %H:%M")):
        df = pandas.read_csv(datalocation, sep=",")

        if Symbol != '':
            df = df[df['Symbol'] == Symbol]
        df['Date'] = pandas.to_datetime(df['Date'] + ' ' + df['Time'])
        
        df['ATR7'] = average_true_range(df["High"], df["Low"], df["Close"], n=7,fillna=True)
        df['ATR14'] = average_true_range(df["High"], df["Low"], df["Close"], n=14,fillna=True)
        df['ATR21'] = average_true_range(df["High"], df["Low"], df["Close"], n=21,fillna=True)
        df['ATR32'] = average_true_range(df["High"], df["Low"], df["Close"], n=32,fillna=True)
         
        df['MACD'] = macd(df['Close'], n_fast=12, n_slow=26, fillna=False)
        df['RSI'] = rsi(df['Close'], n=14, fillna=False)
        
        
        df = df[df['Date'] <= EndDate]
        df = df[df['Date'] >= StartDate]
        df.reset_index()
        sdl = StockDataList()
        #sdl.MountData(df)
        sdl.df = df
        return sdl








   

# Model

In [0]:
class Actor(nn.Module):
    def __init__(self, num_inputs,hidden_size):
      
        super(Actor, self).__init__()
        self.L1hiddenActor=torch.zeros(1,1,hidden_size)
        self.L1memmoryActor=torch.zeros(1,1,hidden_size)
      
        
        self.actorL1 =nn.LSTM(
            input_size=num_inputs,
            hidden_size=hidden_size,
            num_layers=1,
        )

        self.actorLinearL2 = nn.Linear(hidden_size, 2)

    def forward(self, input):
            
        muL1,muStatesL1=self.actorL1(input,(self.L1hiddenActor,self.L1memmoryActor))
        muOutL2 = self.actorLinearL2(muL1)
        
        dist=Categorical(muOutL2)
 
        return muL1,dist
      

class Critic(nn.Module):
    def __init__(self, num_inputs,hidden_size):
        super(Critic, self).__init__()
      
      
        self.L1hiddenCritic=torch.zeros(1,1,hidden_size)
        self.L1memmoryCritic=torch.zeros(1,1,hidden_size)
        

        self.criticL1 = nn.LSTM(
            input_size=num_inputs,
            hidden_size=hidden_size,
            num_layers=1,
        )
        
        self.criticLinearL2 = nn.Linear(hidden_size, 1)


    def forward(self, input):
      
        valueL1 ,valueStatesL1 = self.criticL1(input,(self.L1hiddenCritic,self.L1memmoryCritic))       
        valueOutL2 = self.criticLinearL2(valueL1)


        return valueOutL2
      

# Train

In [0]:
from google.colab import files
uploaded = files.upload()


In [ ]:


# HyperParameters

GAMMA = 0.6   # Cares about Closest reward or future reward 0 myopic 1 farsighted
GAE_LAMBDA = 0.95
PPO_EPSILON = 0.2
CRITIC_DISCOUNT = 1
ENTROPY_BETA = 0.1
PPO_STEPS = 40
MINI_BATCH_SIZE = 16
PPO_EPOCHS = 1
TEST_EPOCHS =10
NUM_TESTS = 1
TARGET_REWARD = 100e6



# ENV parameters

config = EnvironmentConfiguration()
config.DataLocation = 'EURUSD.e15.csv'
config.TrainStartDate = datetime.datetime.strptime('2017/7/5 00:00', "%Y/%m/%d %H:%M") # it should be same with test_env it cant change singlely
config.TrainEndDate = datetime.datetime.strptime('2019/3/1 23:47', "%Y/%m/%d %H:%M")
env = Environment()
env.initializeEnv(config)
TestEnv=copy.deepcopy(env)
print(env.Info)



previousDays=30    # number Of previous Days feed To lstm
dataCount = env.getDataPointCount() - (previousDays + 1)


# Autodetect CUDA
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Device:', device)


# PPO model 

modelActor = Actor(12,100).to(device)
modelCritic = Critic(12,10).to(device)

LEARNING_RATE_ACTOR = 0.05
LEARNING_RATE_CRITIC = 0.01

optimizerActor = optim.Adadelta(modelActor.parameters(), lr = LEARNING_RATE_ACTOR) 
optimizerCritic = optim.Adadelta(modelCritic.parameters(), lr=LEARNING_RATE_CRITIC)


scaler = MinMaxScaler(feature_range=(-1,1))




 

# make Directory for saving Trained Models

def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path




#Returns the log of the probability density/mass function evaluated at value.
def getLogProbFromDist(dist,actions):
    logProbTemp=[]
 
    for action in actions:
        
        
        logProbTemp.append(dist.log_prob(action)[0][0])

    return torch.tensor(logProbTemp)

#  add Last State of enviroment to our state list and remove oldest one
def addLastStateToStateList(states,newState):
    newLSTMStates=[]
    newLSTMStates.append([newState])

    for i in  (range(states.size()[0])):
        if i==states.size()[0]-1:
            break
        newLSTMStates.append(states[i].tolist())

    return torch.tensor(newLSTMStates)

# normalize single Number beetween range 
def singleNormalize(number,_range):
    tempList=[number,-_range,_range]
#     if number>_range :
#       print("warning number is Bigger  than range",' number = ',number ,' range = ',_range)
#     if number<-_range :
#       print("warning number is Smaller than range",' number = ',number ,' range = ',_range)
    numpyTemp = np.asarray([tempList])
    numpyTemp = numpyTemp.reshape(-1, 1)
    scaler.fit(numpyTemp)
    tempList = scaler.transform(numpyTemp)
    return tempList[0][0]

# Normalize OHLC percentages
def normalizePercentages(openPercentage,closePercentage,highPercentage,lowPercentage,_range=0):
    openPercentageNormalize=singleNormalize(openPercentage,_range)
    closePercentageNormalize=singleNormalize(closePercentage,_range)
    highPercentageNormalize=singleNormalize(highPercentage,_range)
    lowPercentageNormalize=singleNormalize(lowPercentage,_range)

    return openPercentageNormalize,closePercentageNormalize,highPercentageNormalize,lowPercentageNormalize
  
# reseting delta Price To seperate Trades from Each Other
def resetDeltaPriceHistory(envStateList):
    envStateToList=envStateList.tolist()
    envStateToListTemp=[]
    for i in range(envStateList.size()[0]):
      envStateToList[i][0][11]=0.0
      
      envStateToListTemp.append(envStateToList[i])
    envStateList=torch.tensor(envStateToListTemp)
    return envStateList  
  
  
  
  
  
# get State Array List in size Of previous Days
def getStateListForLSTM(env,previousDays):
    envStates=[]
    envStatesTemp=[]
    getState_volume=0
    for i in range(previousDays):
        getState, reward, done, _ = env.step(ActionType.Stay)
        tempState=getStateFromEnviroment(getState,env)
        
        getState_volume=getState.stockData.Volume
        tempstate=[tempState]
        envStates.append(tempstate)
    for state in  envStates:
      envStatesTemp.insert(0,state)

    return getState_volume,torch.tensor(envStatesTemp)
  
# Get percentage of part inside whole
def percentage(part, whole):
  return 100 * float(part)/float(whole)


#  get OHLC delta percentage beetwen currentState and pastState
def getOCHLpercentage(currentState,pastState):
    closePercentage=percentage(currentState.stockData.Close-pastState.stockData.Close,pastState.stockData.Close)
    highPercentage=percentage(currentState.stockData.High-pastState.stockData.High,pastState.stockData.Close)
    lowPercentage=percentage(currentState.stockData.Low-pastState.stockData.Low,pastState.stockData.Close)
    openPercentage=percentage(currentState.stockData.Open-pastState.stockData.Open,pastState.stockData.Close)
    
    return openPercentage,closePercentage,highPercentage,lowPercentage

# Normalize and Fit every Single State come From ENV
def getStateFromEnviroment(state, env):
    VolumeDelta = percentage(state.stockData.Volume - env.pastState.stockData.Volume,env.pastState.stockData.Volume)

    ATR7 = percentage(state.stockData.volatility_atr7 - env.pastState.stockData.volatility_atr7,env.pastState.stockData.volatility_atr7)
    ATR7 =ATR7 *3
    
    ATR14= percentage(state.stockData.volatility_atr14 - env.pastState.stockData.volatility_atr14,env.pastState.stockData.volatility_atr14)
    ATR14=ATR14*3.5
    
    ATR21= percentage(state.stockData.volatility_atr21 - env.pastState.stockData.volatility_atr21,env.pastState.stockData.volatility_atr21)
    ATR21=ATR21*4

    ATR32= percentage(state.stockData.volatility_atr32 - env.pastState.stockData.volatility_atr32,env.pastState.stockData.volatility_atr32)
    ATR32=ATR32*5
    
    RSI= percentage(state.stockData.momentum_rsi - env.pastState.stockData.momentum_rsi,env.pastState.stockData.momentum_rsi)
    RSI=RSI/5
    
    
    MACD= percentage(state.stockData.trend_macd - env.pastState.stockData.trend_macd,env.pastState.stockData.trend_macd)
    MACD=MACD/50
    
    ATR7 =singleNormalize(ATR7 ,35)
    ATR14=singleNormalize(ATR14,35)
    ATR21=singleNormalize(ATR21,35)
    ATR32=singleNormalize(ATR32,35)
    
    RSI=singleNormalize(RSI,14)
    
    MACD=singleNormalize(MACD,5)

    
    VolumeDelta = singleNormalize(VolumeDelta, 800)
    openPercentage, closePercentage, highPercentage, lowPercentage=getOCHLpercentage(state, env.pastState)
    openPercentageNormalize, closePercentageNormalize, highPercentageNormalize, lowPercentageNormalize= normalizePercentages(openPercentage,closePercentage,highPercentage,lowPercentage,0.75)
 

    if state.lastBuyStockData.Close == 0:
        priceDelta = 0.0
    else:
        priceDelta = percentage(state.stockData.Close - state.lastBuyStockData.Close,
                                state.lastBuyStockData.Close)
        priceDelta = priceDelta * 6



    if state.HoldStock:
        holdStock = 0.5
    else:
        holdStock = -0.5
    fitState = [openPercentageNormalize, closePercentageNormalize, highPercentageNormalize, lowPercentageNormalize, VolumeDelta ,RSI,MACD,ATR7,ATR14,ATR21,ATR32,holdStock, priceDelta]

    fitState = np.around(np.asarray(fitState), decimals=4)

    fitState = [fitState[0], fitState[1], fitState[2], fitState[3], fitState[4],
                fitState[5],fitState[6], fitState[7], fitState[8], fitState[9],fitState[10],fitState[11],fitState[12]]
    
    
    fitState = [fitState[0], fitState[1], fitState[2], fitState[3], fitState[4],
                fitState[5],fitState[6], fitState[7],fitState[8],fitState[9],fitState[10],fitState[12]]
#     print('O =',fitState[0],'C =',fitState[1],'H =',fitState[2],'L =',fitState[3],'V =',fitState[4],'RSI =',fitState[5],'MACD =',fitState[6],'ATR =',fitState[7],'HS =',fitState[8],'PrD =',fitState[9],'St',fitState[10])
    return fitState


# Change Action Format From Numpy To enum for Env.step(actionType)
def getActionForEnviroment(action,holdStock):
    actionType=getActionFromActionNumpy(action)
    defaultResult = actionType,torch.tensor(action)
#     if actionType==ActionType.Sell and holdStock==False :
#         # print("if 1  ActionType.Sell  holdStock==false")
#         defaultResult= ActionType.Stay,torch.tensor([0])
#     elif actionType==ActionType.Buy and holdStock==True :
#         # print("if 2  ActionType.Buy  holdStock==True")
#         defaultResult= ActionType.Stay,torch.tensor([0])
    # print('inputACtion=',action,'defaultResult',defaultResult,'holdStock=',holdStock)
    return defaultResult


def getActionFromActionNumpy(action):
    if action==[0]:
        return ActionType.Buy
    elif action==[1]:
        return ActionType.Sell



# test ENV is enviroment for test Model performance
def test_env(env, modelActor,modelCritic, device, deterministic=True):
  
#   config ENV to Test Data 
    config = EnvironmentConfiguration()
    config.DataLocation = 'EURUSD.e15.csv'
    config.TrainStartDate = datetime.datetime.strptime('2019/3/1 00:00', "%Y/%m/%d %H:%M") # it should be same with test_env it cant change singlely
    config.TrainEndDate = datetime.datetime.strptime('2019/3/27 23:47', "%Y/%m/%d %H:%M")
    env = Environment()
    env.initializeEnv(config)
    env.reset(1)
    
    
    done = False
    total_reward = 0
#   GetState and prepair for lstm
    lastStateVolumeTest,startStateListTest=getStateListForLSTM(env, previousDays=previousDays)
    torchTestState = startStateListTest
    envStateListTest = startStateListTest
    holdStockTest=False
    
    
    
    for i in range(100):
        torchTestState = torch.FloatTensor(torchTestState).to(device)  # send state to NN

        L1Out,dist= modelActor(torchTestState)
        _= modelCritic(torchTestState)
        
        print('valueOut',_[0][0])

        print('dist.probs',dist.probs[0][0])
        action=torch.argmax(dist.probs[0][0])  # get the action with highest probality
        actiontype,editedAction = getActionForEnviroment(action.cpu().numpy(),holdStockTest)
        print("actiontype",actiontype)
        envNextTestState, reward, done, _ = env.step(actiontype)
        
        envStateListTest = addLastStateToStateList(envStateListTest, getStateFromEnviroment(envNextTestState,env))
        holdStockTest=envNextTestState.HoldStock
        lastStateVolumeTest=envNextTestState.stockData.Volume
        
        print("reward",reward.TotalPoint)
        reward=reward.TotalPoint
        total_reward += reward
          
        if done:
          envStateListTest=resetDeltaPriceHistory(envStateListTest)

        print('\n')
          
        torchTestState = envStateListTest

    print('Test total_reward',total_reward)
    
#   return ENV to train State
    config.TrainStartDate = datetime.datetime.strptime('2017/7/5 00:00', "%Y/%m/%d %H:%M") # it should be same with test_env it cant change singlely
    config.TrainEndDate = datetime.datetime.strptime('2019/3/1 23:47', "%Y/%m/%d %H:%M")
    
    env.initializeEnv(config)
    
    return total_reward


def normalize(x):
    x -= x.mean()
    x /= (x.std() + 1e-1)
    return x

# Compute General Advance Estimation
def compute_gae(next_value, rewards, masks, values, gamma=GAMMA, lam=GAE_LAMBDA):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):

        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * lam * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

# prepare batch data For update Model
def ppo_iter(states, actions, log_probs, returns, advantage):
    for i in range(len(actions)):
        start = i*previousDays
        stop = start + previousDays
        
        
        yield states[start:stop, :], actions[i], log_probs[i], returns[i,:], advantage[i, :]

# update Model
def ppo_update(frame_idx, states, actions, log_probs, returns, advantages, clip_param=PPO_EPSILON):
    count_steps = 0
    sum_returns = 0.0
    sum_advantage = 0.0
    sum_loss_actor = 0.0
    sum_loss_critic = 0.0
    sum_entropy = 0.0
    sum_loss_total = 0.0
    running_loss=0
    # PPO EPOCHS is the number of times we will go through ALL the training data to make updates
    for _ in range(PPO_EPOCHS):
        # grabs batche until we have covered all data
        for state, action, old_log_probs, return_, advantage in ppo_iter(states, actions, log_probs,
                                                                         returns, advantages):
            L1Out,dist = modelActor(state)
            value = modelCritic(state)
            
            entropy = dist.entropy().mean()   # ACTOR
            new_log_probs = dist.log_prob(action)[0][0].to(device) # ACTOR
            
            ratio = (new_log_probs - old_log_probs).exp()  # ACTOR
            
           
          
          
            surr1 = ratio * advantage  # ACTOR & CRITIC 
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - torch.FloatTensor([[value[0][0].item()]])).pow(2).mean().to(device)

            loss = CRITIC_DISCOUNT * critic_loss + actor_loss - ENTROPY_BETA * entropy
            
            # update NN
            
            optimizerActor.zero_grad()  
            optimizerCritic.zero_grad()  
            
            
            loss.backward()
            
            optimizerActor.step()
            optimizerCritic.step()
            
            
            
            
            running_loss += loss.item()
    
    print('running_loss',loss.item())
    

if __name__ == "__main__":
    mkdir('.', 'checkpointsForBorsa')


    print(modelActor)
    print(modelCritic)
    
    frame_idx = 0
    train_epoch = 0
    best_reward = None
    env.reset(1)
    envRange=0
#   Prepare State
    lastStateVolume,startStateList=getStateListForLSTM(env, previousDays=previousDays)
    torchState=startStateList
    envStateList=startStateList
    
    
    holdStock=False
    early_stop = False
    itearationNumber=1
    while not early_stop:

        log_probs = []
        values = []
        states = []
        actions = []
        rewards = []
        masks = []
        actionSeries=0
        
        for _ in range(PPO_STEPS):
            
            if envRange >= (dataCount-previousDays):

                print('env.reset')

                env.reset(itearationNumber)
                itearationNumber+=1
                lastStateVolumeReset, startStateList = getStateListForLSTM(env, previousDays=previousDays)
                envStateList=startStateList
                torchState=startStateList
                envRange = 0
            envRange += 1
            torchState = torch.FloatTensor(torchState).to(device)
            
            L1Out,dist = modelActor(torchState) # get action from Actor
            value = modelCritic(torchState) # get value from Critic
          
            action=torch.argmax(dist.probs[0][0])  # get the action with highest probality
            action=action.item()
            action=torch.tensor([action]).to(device)
         
            
            print('dist.probs',dist.probs[0][0],value[0][0])
            

            actiontype, editedAction = getActionForEnviroment(action.cpu().numpy(),holdStock)

            envNextState, reward, done, __ = env.step(actiontype)  # each state, reward, done 
            holdStock=envNextState.HoldStock

            # append Stuffs

            log_prob = getLogProbFromDist(dist,action)  #Returns the log of the probability density/mass function evaluated at value.
            log_probs.append(log_prob)
            values.append(torch.tensor([[value[0][0].item()]]))
            rewards.append(torch.tensor([[reward.TotalPoint]]))
            states.append(torchState)
            actions.append(editedAction)
            frame_idx += 1
            
            if done:
                masks.append(torch.FloatTensor([0]))
                envStateList=resetDeltaPriceHistory(envStateList)
                torchState = envStateList
                break
            else:
                masks.append(torch.FloatTensor([1]))
                
                
                
            if _>30:
                env.step(ActionType.Sell) 
                envStateList=resetDeltaPriceHistory(envStateList)
                torchState = envStateList
                break
                
                
                
            envStateList=addLastStateToStateList(envStateList,getStateFromEnviroment(envNextState,env))
            lastStateVolume = envNextState.stockData.Volume
            torchState = envStateList
            

            
        # Add next V(s) as 0.0
        returns = compute_gae(torch.tensor([0.0]), rewards, masks, values) #compute general advance estimation
        returns = torch.cat(returns).detach() #returns flat tensor without details

        log_probs = torch.cat(log_probs).detach()
        log_probs=np.around(log_probs,decimals=3)

        values = torch.cat(values)
        states = torch.cat(states)
        actions = torch.cat(actions)

        advantage = returns - values
        advantage = normalize(advantage)
        
        ppo_update(frame_idx, states, actions, log_probs, returns, advantage)
        
#       Empty All Lists
        log_probs = []
        values = []
        states = []
        actions = []
        rewards = []
        masks = []
       
      
      
      
      
#       Go to test model performance and save its better than old Model
        train_epoch += 1
        print('train_epoch',train_epoch)
        if train_epoch % TEST_EPOCHS == 0:
            print('best_reward',best_reward)

            test_reward = np.mean([test_env(TestEnv, modelActor,modelCritic, device) for _ in range(NUM_TESTS)])

            print('Frame %s. reward: %s' % (frame_idx, test_reward))
            # Save a checkpoint every time we achieve a best reward
            if best_reward is None or best_reward < test_reward:
                if best_reward is not None:
                    print("Best reward updated: %.3f -> %.3f" % (best_reward, test_reward))
                    nameAc = "%s_best_%+.3f_%d.dat" % ('Actor', test_reward, frame_idx)
                    nameCr = "%s_best_%+.3f_%d.dat" % ('Critic', test_reward, frame_idx)
                    
                    Actorname = os.path.join('.', 'checkpointsForBorsa', nameAc)
                    Criticname = os.path.join('.', 'checkpointsForBorsa', nameCr)
                    
                    torch.save(modelActor.state_dict(), Actorname)
                    torch.save(modelCritic.state_dict(), Criticname)
                    
                best_reward = test_reward

            if test_reward > TARGET_REWARD: early_stop = True